In [1]:
%run imports_and_functions.ipynb

from dbfread import DBF

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
# income data files come in .dbf, must install this to read from python

# !pip install dbfread

### Creating Final Businesses Dataframe within City Radius

In [2]:
df = pd.read_csv('PhoenixBusinessesUpdated.csv', sep=';', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15386 entries, 0 to 15385
Data columns (total 13 columns):
Unnamed: 0       15386 non-null int64
alias            15386 non-null object
attributes       0 non-null float64
categories       15386 non-null object
coordinates      15386 non-null object
distance         15386 non-null float64
latitude         15315 non-null float64
longitude        15315 non-null float64
main_category    15386 non-null object
name             15386 non-null object
price            4746 non-null object
rating           15386 non-null float64
review_count     15386 non-null int64
dtypes: float64(5), int64(2), object(6)
memory usage: 1.5+ MB


In [8]:
df.loc[[4237, 15365, 14695]] 
# updated dropduplicates below for chain businesses, in multiple same locations

,address,attributes,categories,city,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
4237,1970 W Baseline Rd,"{'BikeParking': 'False', 'BusinessAcceptsCredi...","Coffee & Tea, Food, Restaurants, Cafes",Phoenix,33.379040,-112.101748,Starbucks,NaN,85041.0,15,2.0,AZ
15365,4123 W Thunderbird Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Coffee & Tea, Food",Phoenix,33.610476,-112.147455,Starbucks,NaN,85053.0,13,3.0,AZ
14695,4123 W Thunderbird Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Bakeries, Coffee & Tea, Food",Phoenix,33.610476,-112.147455,Starbucks,NaN,85053.0,6,2.5,AZ


In [3]:
def dropduplicate_yelp_entries(df) :
    dups = df[df[['latitude','longitude','alias']].duplicated(keep=False)].sort_values('latitude')
    mask1 = dups[['latitude','longitude','alias']].duplicated(keep='first')
    mask2 = dups[['latitude','longitude','alias']].duplicated(keep=False)
    mask = mask2 & ~mask1
#     print(dups)
    for index in dups[mask].index :
        dupset = dups[(dups['alias']==dups.loc[index,'alias']) & (dups['longitude']==dups.loc[index,'longitude'])].sort_values(
            'review_count', ascending=False)
        dropindex = dupset.iloc[1:,0].index
        df.drop(dropindex, axis=0, inplace=True)
        
def getPriceRange(string) :
    regex = re.compile("RestaurantsPriceRange2\\': \\'(\d)")
    try :
        match = regex.search(string)
        return int(match.groups(0)[0])
    except : return 0
    
def convertpricecolumn(pricesymbol) :
    try : num = len(pricesymbol)
    except : num = 0
    return num

def get_main_category(catstring) :
    string = ''
    try :
        for cat in main_categories :
            if cat in catstring : string = string + cat + ', '
    except : pass
    return string[0:(len(string)-2)]

In [4]:
print(df.shape)
dropduplicate_yelp_entries(df)
df.shape

(15386, 13)


(15385, 13)

In [5]:
df.head()

,Unnamed: 0,alias,attributes,categories,coordinates,distance,latitude,longitude,main_category,name,price,rating,review_count
0,0,eastlake-park-phoenix,NaN,"[{'alias': 'parks', 'title': 'Parks'}]","{'latitude': 33.4470253, 'longitude': -112.048...",498.636238,33.447025,-112.048897,active,Eastlake Park,NaN,5.0,1
1,1,southwest-autism-research-and-resource-center-...,NaN,"[{'alias': 'preschools', 'title': 'Preschools'...","{'latitude': 33.45181, 'longitude': -112.04386}",799.580740,33.451810,-112.043860,active,Southwest Autism Research & Resource Center,NaN,5.0,3
2,2,mountainside-fitness-phoenix-3,NaN,"[{'alias': 'gyms', 'title': 'Gyms'}, {'alias':...","{'latitude': 33.4463835470996, 'longitude': -1...",2116.538429,33.446384,-112.066437,active,Mountainside Fitness,NaN,3.5,42
3,3,d-backs-race-against-cancer-phoenix,NaN,"[{'alias': 'races', 'title': 'Races & Competit...","{'latitude': 33.4452, 'longitude': -112.06685}",2128.557179,33.445200,-112.066850,active,D-Backs Race Against Cancer,NaN,4.0,1
4,4,chase-field-phoenix,NaN,"[{'alias': 'stadiumsarenas', 'title': 'Stadium...","{'latitude': 33.4452, 'longitude': -112.06685}",2128.557179,33.445200,-112.066850,active,Chase Field,NaN,4.0,650


In [6]:
df['price'] = df['price'].map(convertpricecolumn)
df['price'].value_counts()

0    10640
2     2446
1     1783
3      407
4      109
Name: price, dtype: int64

In [7]:
# df['main_category'] = df['categories'].map(get_main_category)
df['main_category'].value_counts()

homeservices          2847
restaurants           2003
shopping              1952
health                1911
automotive             947
localservices          856
food                   683
professional           650
beautysvc              636
eventservices          519
realestate             359
financialservices      345
active                 300
arts                   287
hotelstravel           268
nightlife              240
education              228
pets                   140
religiousorgs           85
publicservicesgovt      83
massmedia               46
Name: main_category, dtype: int64

In [8]:
# SET FOR EACH CITY: maxradius, testlat, testlong
maxradius = 9000
testlat = 33.444506
testlong = -112.043629 # Phoenix, Arizona

In [9]:
df['distance'] = calcDistances(testlat, testlong, df)

df[df['distance']<maxradius].shape

(10463, 13)

In [10]:
df['distance'].sort_values(ascending=False)

2203     8.290575e+06
6005     8.033615e+06
6846     3.650506e+06
10553    3.189810e+06
10536    3.010308e+06
7249     2.986202e+06
1576     1.822766e+06
8010     1.671697e+06
7349     9.426630e+05
5909     5.965585e+05
14783    4.198717e+05
2835     1.688292e+05
8561     1.602281e+05
2233     6.299739e+04
3884     5.100428e+04
5098     2.353709e+04
2470     1.996457e+04
5011     1.996457e+04
13054    1.792074e+04
11656    1.792074e+04
11657    1.792074e+04
12765    1.687178e+04
12767    1.687178e+04
12766    1.687178e+04
12774    1.687178e+04
12764    1.687178e+04
13427    1.653048e+04
12823    1.644879e+04
12826    1.644879e+04
11517    1.644842e+04
12821    1.644842e+04
12810    1.643280e+04
12816    1.643280e+04
12818    1.643280e+04
12819    1.643280e+04
12828    1.643280e+04
12807    1.643280e+04
12825    1.643280e+04
12809    1.643280e+04
12221    1.625050e+04
15355    1.623329e+04
11213    1.623305e+04
11214    1.623305e+04
15354    1.623085e+04
11209    1.620878e+04
3735     1

In [21]:
df[df[['latitude','longitude']].duplicated(keep=False)].sort_values('latitude')

,Unnamed: 0,alias,attributes,categories,coordinates,distance,latitude,longitude,main_category,name,price,rating,review_count
15384,20810,gamestop-4741-phoenix,NaN,"[{'alias': 'hobbyshops', 'title': 'Hobby Shops'}]","{'latitude': 33.32039, 'longitude': -111.9798}",14992.417654,33.320390,-111.979800,shopping,GameStop #4741,$$$,4.5,9
14074,18551,home-detail-inspections-phoenix-4,NaN,"[{'alias': 'home_inspectors', 'title': 'Home I...","{'latitude': 33.32039, 'longitude': -111.9798}",14992.417654,33.320390,-111.979800,homeservices,Home Detail Inspections,NaN,5.0,3
14075,18552,commercial-builders-alliance-phoenix,NaN,"[{'alias': 'contractors', 'title': 'Contractor...","{'latitude': 33.32039, 'longitude': -111.9798}",14992.417654,33.320390,-111.979800,homeservices,Commercial Builders Alliance,NaN,5.0,1
14076,18553,sierra-vista-painting-phoenix,NaN,"[{'alias': 'painters', 'title': 'Painters'}]","{'latitude': 33.32039, 'longitude': -111.9798}",14992.417654,33.320390,-111.979800,homeservices,Sierra Vista Painting,NaN,5.0,4
14080,18557,anette-brown-real-estate-phoenix,NaN,"[{'alias': 'realestatesvcs', 'title': 'Real Es...","{'latitude': 33.32039, 'longitude': -111.9798}",14992.417654,33.320390,-111.979800,homeservices,Anette Brown Real Estate,NaN,5.0,1
14077,18554,john-horn-custom-house-painting-phoenix,NaN,"[{'alias': 'painters', 'title': 'Painters'}]","{'latitude': 33.32039, 'longitude': -111.9798}",14992.417654,33.320390,-111.979800,homeservices,John Horn Custom House painting,NaN,5.0,1
14079,18556,the-offer-company-phoenix,NaN,"[{'alias': 'realestatesvcs', 'title': 'Real Es...","{'latitude': 33.32039, 'longitude': -111.9798}",14992.417654,33.320390,-111.979800,homeservices,The Offer Company,NaN,5.0,1
14078,18555,legacy-home-management-phoenix,NaN,"[{'alias': 'propertymgmt', 'title': 'Property ...","{'latitude': 33.32039, 'longitude': -111.9798}",14992.417654,33.320390,-111.979800,homeservices,Legacy Home Management,NaN,5.0,1
11217,13724,jasons-deli-chandler,NaN,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...","{'latitude': 33.321276, 'longitude': -111.967601}",15389.856714,33.321276,-111.967601,restaurants,Jason's Deli,$,3.5,127
15357,20778,turn-style-consignment-chandler,NaN,"[{'alias': 'vintage', 'title': 'Used, Vintage ...","{'latitude': 33.321276, 'longitude': -111.967601}",15389.856714,33.321276,-111.967601,shopping,Turn Style Consignment,$,3.5,15


In [11]:
rowstodrop = df[(df['distance']>maxradius) | df['latitude'].isnull() | df['longitude'].isnull()].index

df.drop(rowstodrop, axis=0, inplace=True)
df.shape

(10463, 13)

In [12]:
df['lat_index'] = round(df['latitude'], 3)
df['long_index'] = df['longitude']

df = df.set_index(['lat_index','long_index'], drop=False).sort_index()
df = df.sort_index(level = [0,1])

In [13]:
df.head()

Unnamed: 0                                  alias  \
lat_index long_index                                                       
33.366    -112.065170       13839          maravilla-care-center-phoenix   
33.367    -112.055788       15601             jim-tucker-roofing-phoenix   
33.368    -112.064719       15589  supercool-heating-and-cooling-phoenix   
          -112.049320        1459               the-rag-tacker-phoenix-2   
33.370    -112.055622        7467                 robinson-ranch-phoenix   

                       attributes  \
lat_index long_index                
33.366    -112.065170         NaN   
33.367    -112.055788         NaN   
33.368    -112.064719         NaN   
          -112.049320         NaN   
33.370    -112.055622         NaN   

                                                              categories  \
lat_index long_index                                                       
33.366    -112.065170  [{'alias': 'rehabilitation_center', 'title': '...   
33.367    -112.055788         [{'alias': 'roofing', 'title': 'Roofing'}]   
33.368    -112.064719  [{'alias': 'hvac', 'title': 'Heating & Air Con...   
          -112.049320  [{'alias': 'autoupholstery', 'title': 'Auto Up...   
33.370    -112.055622  [{'alias': 'nonprofit', 'title': 'Community Se...   

                                                             coordinates  \
lat_index long_index                                                       
33.366    -112.065170   {'latitude': 33.366016, 'longitude': -112.06517}   
33.367    -112.055788  {'latitude': 33.366611, 'longitude': -112.055788}   
33.368    -112.064719  {'latitude': 33.368136, 'longitude': -112.064719}   
          -112.049320    {'latitude': 33.36813, 'longitude': -112.04932}   
33.370    -112.055622  {'latitude': 33.3696265319692, 'longitude': -1...   

                          distance   latitude   longitude  main_category  \
lat_index long_index                                                       
33.366    -112.065170  8933.101984  33.366016 -112.065170         health   
33.367    -112.055788  8713.191276  33.366611 -112.055788   homeservices   
33.368    -112.064719  8694.554446  33.368136 -112.064719   homeservices   
          -112.049320  8487.518620  33.368130 -112.049320     automotive   
33.370    -112.055622  8379.608381  33.369627 -112.055622  localservices   

                                              name  price  rating  \
lat_index long_index                                                
33.366    -112.065170        Maravilla Care Center      0     1.0   
33.367    -112.055788           Jim Tucker Roofing      0     5.0   
33.368    -112.064719  SuperCool Heating & Cooling      0     4.0   
          -112.049320               The Rag Tacker      0     5.0   
33.370    -112.055622               Robinson Ranch      0     3.5   

                       review_count  lat_index  long_index  
lat_index long_index                                        
33.366    -112.065170             1     33.366 -112.065170  
33.367    -112.055788             1     33.367 -112.055788  
33.368    -112.064719             4     33.368 -112.064719  
          -112.049320             5     33.368 -112.049320  
33.370    -112.055622             6     33.370 -112.055622

In [14]:
cat_alias_dict = json.load(open('cat_alias_dict.json','r'))

def categories_to_alias2(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    rawlis = list(set(catstring.split(', ')))
    lis = []
    for item in rawlis :
        if 'alias' in item :
            alias = item.split(':')[1].strip().strip("'")
            lis += [alias]
#     print(lis)
    lis = list(map(get_alias2,lis))
    lis = [item for item in lis if item != '_other']
    lis += ['_' + dfrow['main_category']]
    # some new categories are not in category dictionary, and each row already has main category
    # so am taking out all instances where business category could not be found
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias2(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat_alias_dict[cat])])
    except : alias = '_other'
    return alias

def categories_to_alias(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    lis = list(set(catstring.split(', ')))
    lis = map(get_alias,lis)
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat)])
    except : alias = '_other'
    return alias

## Function to map titles to macro-categories if not in top 150

dict_2A = json.load(open('dict_2A.json','r'))
dict_2B = json.load(open('dict_2B.json','r'))

def map_category(category):
    if category not in top_148:
        if category in dict_2B.keys():
            title = dict_2B[category]
        else:
            title = category
        if title in dict_2A.keys():
            title_2 = dict_2A[title]
            return(title_2)
        else:
            return(title)
    else:
        return(category)

In [15]:
df['alias_cats'] = df.apply(categories_to_alias2, axis=1)

In [17]:

df.to_csv('PhoenixBusinessesFinalUpdated.csv',sep=';',header=True,index=True,compression='gzip')

In [16]:
df['alias_cats']

lat_index  long_index 
33.366     -112.065170                                             0_health
33.367     -112.055788                                       0_homeservices
33.368     -112.064719                                0_homeservices 0_hvac
           -112.049320                                  0_auto 0_automotive
33.370     -112.055622                                      0_localservices
33.371     -112.052294                                0_health 0_medcenters
           -112.047606                                      0_localservices
33.372     -112.039800                                       0_homeservices
33.373     -112.032738                       0_localservices 0_professional
           -112.032738                      0_realestateagents 0_realestate
           -112.032738                                       0_homeservices
           -112.032738                                       0_homeservices
           -112.032738                         0_homeservices 0_l

### Census Block Population and Income to X_train Coordinates DataFrame

In [22]:
table = DBF('tl_2014_04_tabblock10.dbf')
type(table)

dbfread.dbf.DBF

In [23]:
df_coords = pd.DataFrame(iter(table))
df_coords.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10
0,04,001,970502,1599,040019705021599,Block 1599,G5040,R,,,S,0,251271,+33.9097858,-109.4197413
1,04,011,960100,1275,040119601001275,Block 1275,G5040,R,,,S,0,8809,+33.1205764,-109.2852833
2,04,023,966101,1103,040239661011103,Block 1103,G5040,R,,,S,3013295,1901,+31.6172066,-111.0172819
3,04,023,966105,1080,040239661051080,Block 1080,G5040,R,,,S,576618,918,+31.3361979,-110.9826473
4,04,023,966105,1068,040239661051068,Block 1068,G5040,R,,,S,36049198,11450,+31.4063471,-111.1182378


In [24]:
df_coords = df_coords[['GEOID10','INTPTLAT10','INTPTLON10','UR10','ALAND10','AWATER10']]
df_coords.to_csv('PhoenixCoords.csv',sep=';',header=True,index=True,compression='gzip')

In [25]:
def cut(geoid) :
    index = str(geoid)
    return int(index[0:(len(index)-3)])

def matchincome(geoid) :
    try : return blockgroupincome.loc[geoid,'income']
    except : return 0

def matchpop(geoid) :
    try : return blockgrouppop.loc[geoid,'population']
    except : return 0

In [26]:
table = DBF('tabblock2010_04_pophu.dbf')

populations = pd.DataFrame(iter(table))
populations.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10
0,04,027,011107,1059,040270111071059,N,0,0
1,04,027,011107,2002,040270111072002,N,0,0
2,04,027,011107,2045,040270111072045,N,0,0
3,04,027,012100,1437,040270121001437,N,0,0
4,04,027,010914,2037,040270109142037,N,0,0


In [27]:
populations = populations[['BLOCKID10','POP10']]

In [28]:
populations.columns = ['GEO.id2','population']
populations['blockgroup'] = populations['GEO.id2'].map(cut)
populations['GEO.id2'] = populations['GEO.id2'].map(int)
populations.set_index(['GEO.id2'],inplace=True,drop=False)
populations.head()

,GEO.id2,population,blockgroup
GEO.id2,,,
40270111071059,40270111071059,0,40270111071
40270111072002,40270111072002,0,40270111072
40270111072045,40270111072045,0,40270111072
40270121001437,40270121001437,0,40270121001
40270109142037,40270109142037,0,40270109142


In [29]:
populations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241666 entries, 40270111071059 to 40250016033039
Data columns (total 3 columns):
GEO.id2       241666 non-null int64
population    241666 non-null int64
blockgroup    241666 non-null int64
dtypes: int64(3)
memory usage: 7.4 MB


In [30]:
blockgroupincome = pd.read_csv('ACS_17_ArizonaIncome.csv')
blockgroupincome.head()

,GEO.id2,population,income
0,Id2,Estimate,NaN
1,40130101011,54499,1818.0
2,40130101012,43901,1758.0
3,40130101013,72350,1339.0
4,40130101021,78229,978.0


In [31]:
blockgroupincome.drop(0, axis=0, inplace=True)
blockgroupincome['GEO.id2'] = blockgroupincome['GEO.id2'].map(int)
blockgroupincome.set_index('GEO.id2', inplace=True)
blockgroupincome.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2503 entries, 40130101011 to 40139807001
Data columns (total 2 columns):
population    2503 non-null object
income        2503 non-null float64
dtypes: float64(1), object(1)
memory usage: 58.7+ KB


In [32]:
populations['income'] = populations['blockgroup'].map(matchincome)
populations['income'].value_counts()

0.0       169406
910.0        810
532.0        787
978.0        672
3289.0       639
2206.0       635
2921.0       622
5942.0       531
4068.0       487
1218.0       429
1731.0       416
1514.0       327
1151.0       321
1085.0       304
2447.0       269
3740.0       260
3332.0       256
1923.0       239
916.0        228
1359.0       227
1775.0       225
1709.0       216
1161.0       206
1506.0       205
2573.0       203
1362.0       199
1630.0       195
2503.0       194
2961.0       189
1804.0       185
3857.0       182
1267.0       177
2903.0       171
4415.0       171
1596.0       170
1123.0       158
1991.0       156
1600.0       154
1822.0       153
1794.0       153
4499.0       152
2092.0       152
7602.0       150
1407.0       150
1862.0       147
1480.0       147
2185.0       147
1811.0       144
1758.0       142
1645.0       141
1052.0       140
3791.0       134
1856.0       133
3133.0       133
1785.0       132
1196.0       128
1507.0       127
1809.0       126
1339.0       1

In [33]:
blockgrouppop = blockgroupincome

populations['blockgrouppop'] = populations['blockgroup'].map(matchpop)
populations['blockgrouppop'].value_counts()

0         168987
17326        787
14746        753
78229        661
18185        639
22581        622
31203        573
3880         531
33868        487
-            419
32150        416
20328        392
18930        301
33053        282
21180        278
19604        269
21896        260
47731        256
21618        239
18767        228
30785        225
16644        203
24263        196
30453        182
18186        172
46435        171
12395        171
31979        171
15586        160
26170        157
39747        154
43373        152
29197        150
24003        150
26288        145
25530        138
12132        134
23799        133
112222       126
20814        122
17091        119
12895        116
43901        114
35941        113
33372        112
28170        111
35455        111
20964        109
25307        108
38521        107
21718        105
36011        104
27142        101
15428        100
32154        100
54499         98
38691         98
34027         97
18003         

### X_train Coordinates DataFrame

In [34]:
features = ['distance','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea','block']

In [35]:
def blocksdropna(df) :
    df.drop(df[df['income']==0].index, inplace=True)
    mask = df['income'].astype(str)=='-'
    df.drop(df[mask].index, inplace=True)
    df.drop(df[df['population']<20].index, inplace=True)  # min 20 people for block to be counted

In [37]:
blocks = pd.read_csv('PhoenixCoords.csv',sep=';',compression='gzip')
blocks.head()

,Unnamed: 0,GEOID10,INTPTLAT10,INTPTLON10,UR10,ALAND10,AWATER10
0,0,40019705021599,33.909786,-109.419741,R,0,251271
1,1,40119601001275,33.120576,-109.285283,R,0,8809
2,2,40239661011103,31.617207,-111.017282,R,3013295,1901
3,3,40239661051080,31.336198,-110.982647,R,576618,918
4,4,40239661051068,31.406347,-111.118238,R,36049198,11450


In [38]:
blocks.drop('Unnamed: 0', axis=1, inplace=True)
blocks.columns = ['GEO.id2','latitude','longitude','urban','landarea','waterarea']
blocks['blockgroup'] = blocks['GEO.id2'].map(cut)
blocks['GEO.id2'] = blocks['GEO.id2'].map(int)
blocks = blocks.set_index('GEO.id2')
blocks.sort_index(inplace=True)
blocks.head()

,latitude,longitude,urban,landarea,waterarea,blockgroup
GEO.id2,,,,,,
40019426001000,36.915534,-109.759147,R,11931369,0,40019426001
40019426001001,36.845741,-109.787031,R,72198969,3218,40019426001
40019426001002,36.812747,-109.784069,R,62351,0,40019426001
40019426001003,36.847153,-109.834610,R,12691,0,40019426001
40019426001004,36.858668,-109.824409,R,662399,0,40019426001


In [39]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241666 entries, 40019426001000 to 40279800061142
Data columns (total 6 columns):
latitude      241666 non-null float64
longitude     241666 non-null float64
urban         241666 non-null object
landarea      241666 non-null int64
waterarea     241666 non-null int64
blockgroup    241666 non-null int64
dtypes: float64(2), int64(3), object(1)
memory usage: 12.9+ MB


In [40]:
blocks = blocks.join(populations[['income','blockgrouppop','population']], how='inner')
blocks.shape

(241666, 9)

In [41]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241666 entries, 40019426001000 to 40279800061142
Data columns (total 9 columns):
latitude         241666 non-null float64
longitude        241666 non-null float64
urban            241666 non-null object
landarea         241666 non-null int64
waterarea        241666 non-null int64
blockgroup       241666 non-null int64
income           241666 non-null float64
blockgrouppop    241666 non-null object
population       241666 non-null int64
dtypes: float64(3), int64(4), object(2)
memory usage: 18.4+ MB


In [42]:
blocksdropna(blocks)
blocks.shape

(40486, 9)

In [43]:
blocks[(blocks['population']!=0) & (blocks['income']==0)] # must drop as don't have y target (income)

,latitude,longitude,urban,landarea,waterarea,blockgroup,income,blockgrouppop,population
GEO.id2,,,,,,,,,


In [44]:
testlat, testlong  # used as center of Phoenix

(33.444506, -112.043629)

In [45]:
blocks['distance_from_city'] = blocks.apply(
    (lambda row : calcDist(row['latitude'],row['longitude'],testlat, testlong)),         
    axis=1)

In [46]:
blocks = blocks[blocks['distance_from_city'] < (maxradius - 1000)]
blocks.shape

(2905, 10)

In [47]:
blocks = blocks.reindex(['distance_from_city','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea'], axis=1)
blocks.head()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
GEO.id2,,,,,,,,,,
40131075001008,7928.005497,U,33.513317,-112.066716,40131075001,914.0,62831,41,31165,0
40131075001009,7869.999940,U,33.512366,-112.068371,40131075001,914.0,62831,78,42086,0
40131075001010,7990.910239,U,33.512483,-112.072116,40131075001,914.0,62831,34,29039,0
40131075001011,7906.213443,U,33.511605,-112.072344,40131075001,914.0,62831,26,25054,0
40131075001012,7785.779121,U,33.511525,-112.068550,40131075001,914.0,62831,75,39960,0


In [48]:
blocks[blocks['distance_from_city'] < (maxradius - 1000)][['population']].sum()

population    294309
dtype: int64

In [49]:
def bootstrapBlocks(blocksdf) :
    df = blocksdf.copy()
    print(df.index)
    for index in blocksdf[blocksdf['population']>100].index :
        iterpop = int(np.ceil(blocksdf.loc[index,'population'] / 100))
        lis = [pd.Series(data=blocksdf.loc[index].values, name=index+(10**15*i),
                        index=blocksdf.columns) for i in range(1,iterpop)]
#         print(len(lis))
        df = df.append(lis) # more efficient and faster to append all at once
    return df

In [50]:
blocks = bootstrapBlocks(blocks)
blocks.shape

Int64Index([40131075001008, 40131075001009, 40131075001010, 40131075001011,
            40131075001012, 40131075001013, 40131075001014, 40131075001016,
            40131075003017, 40131075003018,
            ...
            40131173002005, 40133197041015, 40133197041016, 40133197041018,
            40133197041020, 40133197041021, 40133197041022, 40133197041036,
            40133197041037, 40133197041038],
           dtype='int64', name='GEO.id2', length=2905)


(4401, 10)

In [51]:
blocks.to_csv('PhoenixTrainRowCoords.csv',
          sep=';',
          header=True,
          index=True,
          compression='gzip')

## Vectorize Businesses and Assemble X_train

### Construct Category Corpus with Aliases

In [52]:
lat1000m = -115.161605-(-115.172713)

def makecorpus(df, lat, long) :
    distances = calcDistancesV2(df, lat, long)
    distances = pd.DataFrame(data=distances, index=distances.index, columns=['distances'])
    distances = distances.join(df['alias_cats']) #['alias'] = df.loc[distances.index, 'alias']
    print(distances.shape)
    temp = distances.apply(addproximity, axis=1)
    return ' '.join(temp)

def addproximity(dfrow) :
    w = ('near' if (dfrow.loc['distances'] <= 500) else 'far')
    temp = dfrow.loc['alias_cats'].split(' ')
    temp = list(map((lambda x : (x + '_' + w)), temp))
    return ' '.join(temp)

def calcDistancesV2(multiindexdf, lat, long) :
    minlat = lat - lat1000m
    maxlat = lat + lat1000m
    minlong = long - lat1000m
    maxlong = long + lat1000m
#     multiindexdf = multiindexdf.sort_index(level = [0,1])
    idx = pd.IndexSlice
    df = multiindexdf.loc[idx[minlat:maxlat, minlong:maxlong], ['latitude','longitude']]
    distances = pd.Series(index=df.index)
    for index in df.index :
        try :
            distances[index] = calcDist(lat, long, df.loc[index,'latitude'].values[0], 
                                    df.loc[index,'longitude'].values[0])
        except : # catch very rare attribute error to fill in rest of rows
            distances[index] = 10000 # if calcDist gives error, don't include this business
    distances = distances[distances<1000]
    return distances

def make_X_train(coorddf, businessesdf) :
    for index in coorddf.index :
            coorddf.loc[index,'corpus'] = makecorpus(businessesdf, 
                                            coorddf.loc[index,'latitude'], 
                                            coorddf.loc[index,'longitude'])

In [54]:
df = pd.read_csv('PhoenixBusinessesFinalUpdated.csv',sep=';',compression='gzip',
                 index_col=['lat_index.1','long_index.1'])
df.head()

lat_index  long_index  Unnamed: 0  \
lat_index.1 long_index.1                                      
33.366      -112.065170      33.366 -112.065170       13839   
33.367      -112.055788      33.367 -112.055788       15601   
33.368      -112.064719      33.368 -112.064719       15589   
            -112.049320      33.368 -112.049320        1459   
33.370      -112.055622      33.370 -112.055622        7467   

                                                          alias  attributes  \
lat_index.1 long_index.1                                                      
33.366      -112.065170           maravilla-care-center-phoenix         NaN   
33.367      -112.055788              jim-tucker-roofing-phoenix         NaN   
33.368      -112.064719   supercool-heating-and-cooling-phoenix         NaN   
            -112.049320                the-rag-tacker-phoenix-2         NaN   
33.370      -112.055622                  robinson-ranch-phoenix         NaN   

                                                                 categories  \
lat_index.1 long_index.1                                                      
33.366      -112.065170   [{'alias': 'rehabilitation_center', 'title': '...   
33.367      -112.055788          [{'alias': 'roofing', 'title': 'Roofing'}]   
33.368      -112.064719   [{'alias': 'hvac', 'title': 'Heating & Air Con...   
            -112.049320   [{'alias': 'autoupholstery', 'title': 'Auto Up...   
33.370      -112.055622   [{'alias': 'nonprofit', 'title': 'Community Se...   

                                                                coordinates  \
lat_index.1 long_index.1                                                      
33.366      -112.065170    {'latitude': 33.366016, 'longitude': -112.06517}   
33.367      -112.055788   {'latitude': 33.366611, 'longitude': -112.055788}   
33.368      -112.064719   {'latitude': 33.368136, 'longitude': -112.064719}   
            -112.049320     {'latitude': 33.36813, 'longitude': -112.04932}   
33.370      -112.055622   {'latitude': 33.3696265319692, 'longitude': -1...   

                             distance   latitude   longitude  main_category  \
lat_index.1 long_index.1                                                      
33.366      -112.065170   8933.101984  33.366016 -112.065170         health   
33.367      -112.055788   8713.191276  33.366611 -112.055788   homeservices   
33.368      -112.064719   8694.554446  33.368136 -112.064719   homeservices   
            -112.049320   8487.518620  33.368130 -112.049320     automotive   
33.370      -112.055622   8379.608381  33.369627 -112.055622  localservices   

                                                 name  price  rating  \
lat_index.1 long_index.1                                               
33.366      -112.065170         Maravilla Care Center      0     1.0   
33.367      -112.055788            Jim Tucker Roofing      0     5.0   
33.368      -112.064719   SuperCool Heating & Cooling      0     4.0   
            -112.049320                The Rag Tacker      0     5.0   
33.370      -112.055622                Robinson Ranch      0     3.5   

                          review_count             alias_cats  
lat_index.1 long_index.1                                       
33.366      -112.065170              1               0_health  
33.367      -112.055788              1         0_homeservices  
33.368      -112.064719              4  0_homeservices 0_hvac  
            -112.049320              5    0_auto 0_automotive  
33.370      -112.055622              6        0_localservices

In [55]:
df.describe()

,lat_index,long_index,Unnamed: 0,attributes,distance,latitude,longitude,price,rating,review_count
count,10463.000000,10463.000000,10463.000000,0.0,10463.000000,10463.000000,10463.000000,10463.000000,10463.000000,10463.000000
mean,33.465773,-112.045039,6325.340055,NaN,5627.730841,33.465774,-112.045039,0.458759,3.611918,33.647615
std,0.035939,0.041324,4342.540624,NaN,2134.111531,0.035934,0.041324,0.829694,1.351732,116.757792
min,33.366000,-112.139860,0.000000,NaN,156.817421,33.366016,-112.139860,0.000000,1.000000,1.000000
25%,33.448000,-112.074180,2810.500000,NaN,3683.283014,33.447565,-112.074180,0.000000,2.500000,1.000000
50%,33.466000,-112.049599,5763.000000,NaN,5888.747718,33.466412,-112.049599,0.000000,4.000000,4.000000
75%,33.495000,-112.012800,9274.500000,NaN,7438.646929,33.495342,-112.012800,1.000000,5.000000,17.000000
max,33.525000,-111.947930,20687.000000,NaN,8999.203664,33.525456,-111.947930,4.000000,5.000000,2562.000000


In [20]:
blocks = pd.read_csv('PhoenixTrainRowCoords.csv',
          sep=';', compression='gzip', index_col='Unnamed: 0')
blocks.head()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
40131075001008,7928.005497,U,33.513317,-112.066716,40131075001,914.0,62831,41,31165,0
40131075001009,7869.999940,U,33.512366,-112.068371,40131075001,914.0,62831,78,42086,0
40131075001010,7990.910239,U,33.512483,-112.072116,40131075001,914.0,62831,34,29039,0
40131075001011,7906.213443,U,33.511605,-112.072344,40131075001,914.0,62831,26,25054,0
40131075001012,7785.779121,U,33.511525,-112.068550,40131075001,914.0,62831,75,39960,0


In [56]:
blocks.describe()

,distance_from_city,latitude,longitude,blockgroup,income,population,landarea,waterarea
count,4401.000000,4401.000000,4401.000000,4.401000e+03,4401.000000,4401.000000,4.401000e+03,4401.000000
mean,5480.745577,33.454189,-112.048396,4.013113e+10,1782.973642,236.312429,6.131909e+04,60.864122
std,1774.781953,0.038378,0.039847,1.020109e+05,880.181676,477.168501,9.429418e+04,1089.357585
min,255.437401,33.372665,-112.128954,4.013108e+10,244.000000,20.000000,9.860000e+02,0.000000
25%,4259.907085,33.427499,-112.080254,4.013111e+10,1184.000000,53.000000,2.030400e+04,0.000000
50%,5725.928813,33.463233,-112.049744,4.013113e+10,1634.000000,103.000000,3.033600e+04,0.000000
75%,7028.219375,33.484676,-112.018325,4.013116e+10,2211.000000,237.000000,6.146600e+04,0.000000
max,7998.917110,33.515992,-111.962005,4.013320e+10,5654.000000,4320.000000,1.335537e+06,34088.000000


In [57]:
make_X_train(blocks, df)

(1026, 2)
(1035, 2)
(962, 2)
(948, 2)
(1051, 2)
(1145, 2)
(1049, 2)
(1009, 2)
(1053, 2)
(1089, 2)
(558, 2)
(765, 2)
(607, 2)
(543, 2)
(549, 2)
(540, 2)
(535, 2)
(550, 2)
(545, 2)
(611, 2)
(583, 2)
(519, 2)
(586, 2)
(538, 2)
(553, 2)
(676, 2)
(573, 2)
(512, 2)
(440, 2)
(504, 2)
(661, 2)
(587, 2)
(560, 2)
(538, 2)
(482, 2)
(627, 2)
(635, 2)
(859, 2)
(616, 2)
(681, 2)
(655, 2)
(615, 2)
(1431, 2)
(1589, 2)
(615, 2)
(733, 2)
(1185, 2)
(1194, 2)
(1201, 2)
(1230, 2)
(757, 2)
(854, 2)
(1325, 2)
(896, 2)
(526, 2)
(512, 2)
(778, 2)
(618, 2)
(597, 2)
(659, 2)
(725, 2)
(741, 2)
(716, 2)
(601, 2)
(570, 2)
(565, 2)
(557, 2)
(574, 2)
(575, 2)
(520, 2)
(521, 2)
(520, 2)
(568, 2)
(604, 2)
(649, 2)
(562, 2)
(583, 2)
(473, 2)
(490, 2)
(535, 2)
(605, 2)
(552, 2)
(458, 2)
(572, 2)
(540, 2)
(312, 2)
(441, 2)
(443, 2)
(489, 2)
(240, 2)
(302, 2)
(464, 2)
(371, 2)
(716, 2)
(859, 2)
(556, 2)
(617, 2)
(661, 2)
(595, 2)
(775, 2)
(621, 2)
(563, 2)
(558, 2)
(568, 2)
(573, 2)
(594, 2)
(581, 2)
(592, 2)
(622, 2)
(637

(223, 2)
(365, 2)
(369, 2)
(361, 2)
(378, 2)
(371, 2)
(361, 2)
(372, 2)
(360, 2)
(270, 2)
(360, 2)
(329, 2)
(315, 2)
(288, 2)
(246, 2)
(226, 2)
(247, 2)
(108, 2)
(104, 2)
(105, 2)
(149, 2)
(150, 2)
(103, 2)
(100, 2)
(94, 2)
(97, 2)
(111, 2)
(245, 2)
(249, 2)
(105, 2)
(105, 2)
(97, 2)
(94, 2)
(102, 2)
(131, 2)
(117, 2)
(166, 2)
(168, 2)
(168, 2)
(176, 2)
(136, 2)
(133, 2)
(137, 2)
(132, 2)
(120, 2)
(122, 2)
(110, 2)
(108, 2)
(119, 2)
(436, 2)
(417, 2)
(416, 2)
(412, 2)
(404, 2)
(409, 2)
(391, 2)
(401, 2)
(398, 2)
(433, 2)
(156, 2)
(186, 2)
(385, 2)
(405, 2)
(392, 2)
(414, 2)
(406, 2)
(410, 2)
(166, 2)
(320, 2)
(309, 2)
(162, 2)
(175, 2)
(431, 2)
(170, 2)
(164, 2)
(292, 2)
(287, 2)
(311, 2)
(307, 2)
(306, 2)
(308, 2)
(298, 2)
(292, 2)
(294, 2)
(180, 2)
(170, 2)
(300, 2)
(308, 2)
(312, 2)
(310, 2)
(316, 2)
(312, 2)
(271, 2)
(258, 2)
(259, 2)
(257, 2)
(275, 2)
(311, 2)
(308, 2)
(297, 2)
(290, 2)
(135, 2)
(157, 2)
(139, 2)
(136, 2)
(111, 2)
(116, 2)
(293, 2)
(284, 2)
(460, 2)
(461, 2)
(469,

(12770, 2)
(26603, 2)
(252, 2)
(369, 2)
(230, 2)
(336, 2)
(165, 2)
(104, 2)
(119, 2)
(112, 2)
(109, 2)
(101, 2)
(71, 2)
(62, 2)
(65, 2)
(74, 2)
(71, 2)
(64, 2)
(56, 2)
(62, 2)
(64, 2)
(49, 2)
(57, 2)
(174, 2)
(172, 2)
(142, 2)
(144, 2)
(141, 2)
(130, 2)
(117, 2)
(130, 2)
(136, 2)
(140, 2)
(133, 2)
(111, 2)
(61, 2)
(60, 2)
(57, 2)
(66, 2)
(60, 2)
(65, 2)
(57, 2)
(64, 2)
(62, 2)
(64, 2)
(62, 2)
(192, 2)
(96, 2)
(91, 2)
(67, 2)
(65, 2)
(65, 2)
(86, 2)
(67, 2)
(52, 2)
(55, 2)
(52, 2)
(58, 2)
(55, 2)
(55, 2)
(167, 2)
(113, 2)
(80, 2)
(79, 2)
(81, 2)
(112, 2)
(139, 2)
(146, 2)
(112, 2)
(83, 2)
(82, 2)
(83, 2)
(114, 2)
(128, 2)
(111, 2)
(74, 2)
(71, 2)
(77, 2)
(115, 2)
(61, 2)
(58, 2)
(57, 2)
(68, 2)
(64, 2)
(65, 2)
(66, 2)
(57, 2)
(57, 2)
(56, 2)
(60, 2)
(65, 2)
(61, 2)
(186, 2)
(190, 2)
(198, 2)
(343, 2)
(353, 2)
(189, 2)
(187, 2)
(226, 2)
(347, 2)
(197, 2)
(227, 2)
(235, 2)
(189, 2)
(193, 2)
(360, 2)
(352, 2)
(352, 2)
(363, 2)
(347, 2)
(340, 2)
(343, 2)
(100, 2)
(102, 2)
(103, 2)
(106, 2)


(93, 2)
(81, 2)
(92, 2)
(71, 2)
(62, 2)
(57, 2)
(55, 2)
(57, 2)
(70, 2)
(68, 2)
(69, 2)
(71, 2)
(79, 2)
(58, 2)
(71, 2)
(73, 2)
(56, 2)
(64, 2)
(70, 2)
(72, 2)
(284, 2)
(941, 2)
(929, 2)
(920, 2)
(862, 2)
(842, 2)
(848, 2)
(785, 2)
(785, 2)
(492, 2)
(383, 2)
(415, 2)
(400, 2)
(393, 2)
(353, 2)
(449, 2)
(871, 2)
(906, 2)
(873, 2)
(498, 2)
(455, 2)
(492, 2)
(498, 2)
(425, 2)
(339, 2)
(346, 2)
(452, 2)
(458, 2)
(413, 2)
(436, 2)
(75, 2)
(74, 2)
(71, 2)
(92, 2)
(109, 2)
(109, 2)
(84, 2)
(72, 2)
(90, 2)
(56, 2)
(97, 2)
(104, 2)
(83, 2)
(76, 2)
(84, 2)
(85, 2)
(76, 2)
(93, 2)
(89, 2)
(85, 2)
(81, 2)
(92, 2)
(107, 2)
(103, 2)
(98, 2)
(110, 2)
(76, 2)
(74, 2)
(49, 2)
(50, 2)
(53, 2)
(42, 2)
(43, 2)
(45, 2)
(44, 2)
(41, 2)
(41, 2)
(29, 2)
(31, 2)
(52, 2)
(53, 2)
(48, 2)
(56, 2)
(234, 2)
(224, 2)
(215, 2)
(224, 2)
(209, 2)
(224, 2)
(253, 2)
(252, 2)
(228, 2)
(558, 2)
(765, 2)
(765, 2)
(573, 2)
(573, 2)
(661, 2)
(661, 2)
(661, 2)
(661, 2)
(661, 2)
(661, 2)
(661, 2)
(661, 2)
(661, 2)
(616, 2)
(616

(78, 2)
(78, 2)
(78, 2)
(78, 2)
(57, 2)
(57, 2)
(57, 2)
(57, 2)
(57, 2)
(57, 2)
(57, 2)
(57, 2)
(57, 2)
(54, 2)
(54, 2)
(55, 2)
(70, 2)
(90, 2)
(90, 2)
(90, 2)
(90, 2)
(90, 2)
(90, 2)
(90, 2)
(90, 2)
(90, 2)
(90, 2)
(107, 2)
(107, 2)
(107, 2)
(107, 2)
(107, 2)
(107, 2)
(107, 2)
(107, 2)
(107, 2)
(99, 2)
(99, 2)
(99, 2)
(99, 2)
(101, 2)
(106, 2)
(106, 2)
(106, 2)
(106, 2)
(104, 2)
(97, 2)
(86, 2)
(85, 2)
(85, 2)
(85, 2)
(59, 2)
(59, 2)
(54, 2)
(54, 2)
(54, 2)
(54, 2)
(44, 2)
(63, 2)
(52, 2)
(50, 2)
(193, 2)
(193, 2)
(182, 2)
(182, 2)
(149, 2)
(149, 2)
(149, 2)
(149, 2)
(149, 2)
(218, 2)
(218, 2)
(209, 2)
(214, 2)
(197, 2)
(197, 2)
(197, 2)
(197, 2)
(197, 2)
(197, 2)
(197, 2)
(197, 2)
(220, 2)
(191, 2)
(191, 2)
(191, 2)
(191, 2)
(180, 2)
(180, 2)
(180, 2)
(182, 2)
(182, 2)
(182, 2)
(182, 2)
(182, 2)
(133, 2)
(133, 2)
(133, 2)
(133, 2)
(133, 2)
(133, 2)
(133, 2)
(133, 2)
(133, 2)
(133, 2)
(108, 2)
(108, 2)
(108, 2)
(108, 2)
(108, 2)
(108, 2)
(182, 2)
(182, 2)
(182, 2)
(182, 2)
(182, 2)
(1

In [ ]:
blocks.to_csv('CorpusPhoenixTrainRowCoords.csv',sep=';',compression='gzip')

In [61]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4401 entries, 40131075001008 to 1040133197041038
Data columns (total 11 columns):
distance_from_city    4401 non-null float64
urban                 4401 non-null object
latitude              4401 non-null float64
longitude             4401 non-null float64
blockgroup            4401 non-null int64
income                4401 non-null float64
blockgrouppop         4401 non-null object
population            4401 non-null int64
landarea              4401 non-null int64
waterarea             4401 non-null int64
corpus                4401 non-null object
dtypes: float64(4), int64(4), object(3)
memory usage: 572.6+ KB


In [58]:
X_features = joblib.load('X_features.joblib')

vect = CountVectorizer(dtype='uint16',min_df=1, vocabulary=X_features,
                       decode_error='ignore', lowercase=True) 

X_sparse = vect.transform(blocks['corpus'])

In [59]:
dfvect = pd.DataFrame(X_sparse.toarray(),dtype='uint16',columns=X_features)
dfvect.head()

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
0,0,0,0,5,4,10,6,25,0,0,0,0,5,25,3,5,7,46,2,11,0,21,0,32,2,11,1,11,0,4,0,0,8,6,2,6,14,47,0,1,0,0,12,53,0,...,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,0,0,0
1,0,0,0,5,6,8,9,21,0,0,0,0,17,14,4,6,19,38,8,9,1,20,10,22,4,14,6,7,0,4,0,0,8,4,2,6,13,47,1,0,0,0,21,44,0,...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,0,0,0
2,0,0,0,5,5,4,15,11,0,0,0,0,16,9,3,7,19,30,10,7,20,0,31,2,8,7,9,4,4,0,0,0,1,9,1,7,24,26,1,0,0,0,48,14,0,...,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0
3,0,0,0,1,5,4,15,10,0,0,0,0,17,8,4,6,21,20,10,7,20,0,31,2,8,6,9,2,4,0,0,0,1,11,1,3,25,23,1,0,0,0,48,16,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0
4,0,0,1,4,6,8,12,19,0,0,0,0,17,14,4,6,22,38,8,9,1,20,10,22,3,15,6,6,0,4,0,0,7,5,2,6,13,47,1,0,0,0,21,44,0,...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,0,0,0


In [60]:
joblib.dump(dfvect, 'X_trainPhoenix.joblib', compress=9)

['X_trainPhoenix.joblib']

In [47]:
y_trainPhoenix = blocks['income']

In [48]:
joblib.dump(y_trainPhoenix, 'y_trainPhoenix.joblib', compress=9)

['y_trainPhoenix.joblib']

In [62]:
dfvect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4401 entries, 0 to 4400
Columns: 1482 entries, landarea to 4_physicaltherapy_far
dtypes: uint16(1482)
memory usage: 12.4 MB
